In [ ]:
import logging
from pyspark.sql import SparkSession
from classify_amazonreviews_sparkml import run_pipeline

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

spark = SparkSession.builder \
    .master("local[4]") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

start_date = '2014-01-01'
split_date = '2014-12-01'

run_pipeline(spark, start_date, split_date)

In [ ]:
from freamon.adapters.pyspark.provenance import from_trace

view_gen = from_trace()

In [ ]:
#df = view_gen.test_view(
#    sliceable_by=['category', 'category_id'], 
#    with_features=False, 
#    with_y_true=True, 
#    with_y_pred=True)
#df